# Análise Populacional usando PySCF

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyscf
from pyscf import scf
from pyscf.tools import cubegen # para exportar aquivos cube

## Molécula de H2O

In [2]:
l0 = 0.9578 # comprimento da ligação O-H em Angstroms
angle = 104.49 * (np.pi / 180) # ângulo H

mol = pyscf.M(
    atom=f'O 0 0 0; H 0 {-l0*np.sin(angle/2)} {l0*np.cos(angle/2)}; H 0. {l0*np.sin(angle/2)} {l0*np.cos(angle/2)}',
    basis = 'aug-cc-pVTZ', unit='Angstrom', verbose=0)

Realizando um cálculo de Hartree-Fock restrito (elétrons emparelhados)

In [3]:
mf = scf.RHF(mol).to_gpu()
mf.kernel()

print('Total energy', mf.energy_tot())

Total energy -76.06057190671464


In [4]:
from gpu4pyscf.cc import ccsd_incore

mf.with_df = None   # DF CCSD is not supported yet.
mycc = ccsd_incore.CCSD(mf)

mycc.kernel()

print('CCSD total energy', mycc.e_tot)

CCSD total energy -76.34876077197379


Energias dos orbitais e cargas de mulliken

In [5]:
mf.analyze()

((array([ 1.99999298e+00,  1.64357790e+00,  3.41285009e-03,  1.53122939e-03,
          2.34574423e-04,  1.98369211e+00,  1.34419855e+00,  1.68176399e+00,
          3.50165013e-03,  6.04179219e-03,  8.04353878e-04,  7.84710046e-04,
          1.14250736e-03,  1.77374999e-03,  2.43737551e-04,  4.75612057e-05,
          1.83302372e-04,  2.94002749e-20,  5.99117433e-03,  2.94615311e-03,
          2.60663366e-03,  2.26299724e-03, -9.16353359e-21,  7.28191201e-06,
          4.95928851e-04,  4.41786846e-04,  6.64056374e-05,  1.07131223e-20,
          3.19948024e-04,  2.88302583e-05,  2.55269491e-05,  2.27289846e-04,
          2.70948879e-04,  2.25230051e-21,  7.68498388e-06,  1.64677550e-04,
          4.10056844e-05,  1.44515750e-03,  3.97836638e-04,  1.33364097e-05,
          5.77134334e-21,  8.53592415e-06,  1.89829500e-05,  3.88029883e-06,
          1.75566504e-04,  9.82641228e-05,  6.31890987e-01,  7.77566392e-03,
          4.44783462e-03,  2.13334440e-04,  3.70855572e-03,  1.95194213e-03,

Calculando a matriz densidade

In [6]:
dm = mf.to_cpu().make_rdm1()  # to_cpu

Exportando a densidade eletrônica

In [7]:
# electron density
cubegen.density(mol, 'water_rho.cube', dm)

array([[[1.74511463e-06, 1.89840123e-06, 2.06259603e-06, ...,
         2.20499231e-06, 2.01033175e-06, 1.82861746e-06],
        [2.04293101e-06, 2.22628744e-06, 2.42320356e-06, ...,
         2.52657967e-06, 2.30167823e-06, 2.09209515e-06],
        [2.39030563e-06, 2.60989712e-06, 2.84638286e-06, ...,
         2.88611003e-06, 2.62680365e-06, 2.38562654e-06],
        ...,
        [2.39030563e-06, 2.60989712e-06, 2.84638286e-06, ...,
         2.88611003e-06, 2.62680365e-06, 2.38562654e-06],
        [2.04293101e-06, 2.22628744e-06, 2.42320356e-06, ...,
         2.52657967e-06, 2.30167823e-06, 2.09209515e-06],
        [1.74511463e-06, 1.89840123e-06, 2.06259603e-06, ...,
         2.20499231e-06, 2.01033175e-06, 1.82861746e-06]],

       [[1.84156499e-06, 2.00475688e-06, 2.17975656e-06, ...,
         2.36030123e-06, 2.15088973e-06, 1.95557594e-06],
        [2.15970131e-06, 2.35545038e-06, 2.56592985e-06, ...,
         2.70667485e-06, 2.46436766e-06, 2.23879426e-06],
        [2.53183777e-06, 

Exportando potencial eletrostático

In [8]:
cubegen.mep(mol, 'water_esp.cube', dm)

array([[[-0.00913187, -0.00901328, -0.00888132, ...,  0.01308902,
          0.01298159,  0.01286866],
        [-0.00961705, -0.00950443, -0.00937792, ...,  0.01339144,
          0.01327698,  0.01315697],
        [-0.01012672, -0.01002115, -0.00990119, ...,  0.01369127,
          0.01356975,  0.01344266],
        ...,
        [-0.01012672, -0.01002115, -0.00990119, ...,  0.01369127,
          0.01356975,  0.01344266],
        [-0.00961705, -0.00950443, -0.00937792, ...,  0.01339144,
          0.01327698,  0.01315697],
        [-0.00913187, -0.00901328, -0.00888132, ...,  0.01308902,
          0.01298159,  0.01286866]],

       [[-0.00921107, -0.00908936, -0.00895384, ...,  0.01343088,
          0.01331249,  0.01318877],
        [-0.00970538, -0.00958974, -0.00945976, ...,  0.01374927,
          0.01362302,  0.01349145],
        [-0.01022505, -0.01011659, -0.00999328, ...,  0.01406543,
          0.01393128,  0.0137918 ],
        ...,
        [-0.01022505, -0.01011659, -0.00999328, ...,  

## Análise de Bader

Deve instalar o programa de https://theory.cm.utexas.edu/henkelman/code/bader/

In [9]:
!bader water_rho.cube


  GRID BASED BADER ANALYSIS  (Version 1.05 08/19/23)

  OPEN ... water_rho.cube      
  GAUSSIAN-STYLE INPUT FILE
  DENSITY-GRID:   80 x  80 x  80
  CLOSE ... water_rho.cube      
  RUN TIME:    0.08 SECONDS

  CALCULATING BADER CHARGE DISTRIBUTION
                 0  10  25  50  75  100
  PERCENT DONE:  ********************** 

  REFINING AUTOMATICALLY
  ITERATION: 1
  EDGE POINTS:         57385
  REASSIGNED POINTS:    5809

  RUN TIME:       0.43 SECONDS

  CALCULATING MINIMUM DISTANCES TO ATOMS
                 0  10  25  50  75  100
  PERCENT DONE:  **********************
  RUN TIME:    0.05 SECONDS

  WRITING BADER ATOMIC CHARGES TO ACF.dat
  WRITING BADER VOLUME CHARGES TO BCF.dat

  NUMBER OF BADER MAXIMA FOUND:              4
      SIGNIFICANT MAXIMA FOUND:              3
                 VACUUM CHARGE:         0.0000
           NUMBER OF ELECTRONS:        9.93656
 


In [10]:
Z = np.array([8,1,1])
N = np.array([9.249086,0.343737,0.343737]).round(3)  # from ACF.dat -> Bader analysis

N.sum()

Q = Z - N
print('Number of electrons from Bader analysis:', N)
print("Bader charges (e):", Q)

Number of electrons from Bader analysis: [9.249 0.344 0.344]
Bader charges (e): [-1.249  0.656  0.656]


## Análise RESP

Baseado em: https://github.com/pyscf/gpu4pyscf/blob/master/examples/22-resp_charge.py

Algoritmo de referência é https://onlinelibrary.wiley.com/doi/epdf/10.1002/qua.26035

In [11]:
from gpu4pyscf.pop import esp

# ESP charge
q0 = esp.esp_solve(mol, dm)
print('Fitted ESP charge')
print(q0)

Fitted ESP charge
[-0.72773379  0.3638669   0.3638669 ]


In [24]:
# RESP charge // first stage fitting
q1 = esp.resp_solve(mol, dm)

# Constraints for sum of charges
# sum_constraints = [
#     [c0, [i,j,k,l]],
#     [c1, [i,j,l]]]
#     --> 
#     c0 = q[i] + q[j] + q[k] + q[l]
#     c1 = q[i] + q[j] + q[l]
sum_constraints = [[-q1[0],[1,2]]]  # total charge of H2O is zero

# Constraint for equal charges
# equal_constraints = [
#     [i,j,k],
#     [u,v,w]]
#     -->
#     q[i] = q[j] = q[k] = q[l]
#     q[u] = q[v] = q[w]
equal_constraints = [[1,2]] # H atoms in H2O are equal

# RESP charge // second stage fitting
q2 = esp.resp_solve(mol, dm, resp_a=1e-3,
                    sum_constraints=sum_constraints,
                    equal_constraints=equal_constraints)
print('Fitted partial charge with RESP')
print(q2)

Fitted partial charge with RESP
[-0.72569838  0.36284919  0.36284919]


## Análise CHELPG

Baseado em: https://github.com/pyscf/gpu4pyscf/blob/master/examples/15-chelpg.py

In [25]:
from gpu4pyscf.qmmm import chelpg

q = chelpg.eval_chelpg_layer_gpu(mf)

print('Fitted partial charge with CHELPG')
print(q) 

Fitted partial charge with CHELPG
[-0.72820174  0.36403436  0.36416738]


In [26]:
# Podemos customizar os raios van der Waals usados no CHELPG
from pyscf.data import radii

q = chelpg.eval_chelpg_layer_gpu(mf, Rvdw=radii.UFF)

print('Fitted partial charge with CHELPG, using UFF radii')
print(q)

Fitted partial charge with CHELPG, using UFF radii
[-0.72165506  0.36071157  0.36094348]
